In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2023 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck P. Vidal (UKRI-STFC)

In [ ]:
path = "/DATA/2023/DTHE/ZrO2-Cu-1mm-10umvx"
# path = "/DATA/2023/DTHE/Wire-Cu-2mm-17.54umvx"

In [ ]:
import os
import time

from cil.io import TIFFWriter
from cil.processors import TransmissionAbsorptionConverter, Slicer
from cil.recon import FDK
from cil.plugins.astra import FBP as FBP_astra
from cil.plugins.tigre import FBP as FBP_tigre

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.algorithms import CGLS, SIRT

from DTHEDataReader import *
from FDK import reconstruct, getRuntime

## Load the data

In [ ]:
filename = os.path.join(path, "unireconstruction.xml")

reader = DTHEDataReader(file_name=filename)
data = reader.read()

## Inspect the geometry

Checkout what the data looks like

In [ ]:
print(data)

Checkout what the acquisition geometry looks like

In [ ]:
print(data.geometry)

CIL can even plot what the geometry looks like

In [ ]:
show_geometry(data.geometry).save("geometry.png")

## Inspect the projections

In [ ]:
islicer(data, direction='angle', origin="upper-left")

Apply the minus log

In [ ]:
data_corr = TransmissionAbsorptionConverter()(data)

Inspect the projections

In [ ]:
islicer(data_corr, direction='angle', origin="upper-left")

In [ ]:
roi_size = None#[400, 400, 300]

## FDK using CIL filter and Tigre projector

In [ ]:
start = time.time()

print("Filter: CIL")
print("Projector: Tigre")
# Prepare the data for Tigre
data_corr.reorder(order='tigre')

# Reconstruct using FDK
ig = data_corr.geometry.get_ImageGeometry()

if roi_size is not None:
    ig.voxel_num_x = roi_size[0]
    ig.voxel_num_y = roi_size[1]
    ig.voxel_num_z = roi_size[2]

reconstruction_algorithm =  FDK(data_corr, ig)
reconstruction_algorithm.set_filter_inplace(False)
recons_FDK_cil = reconstruction_algorithm.run()

stop = time.time()
runtime, unit = getRuntime(start, stop)
print("Execution time:", "{0:0.2f}".format(runtime), unit)

We can save the reconstructed volume to disk for example as a stack of TIFFs:

In [ ]:
save_base_path = os.getcwd()
save_path = os.path.join(path, "recons_FDK_cil")
print("Print the CT data will be saved in:", save_path)

if not os.path.isdir(save_path):
    os.makedirs(save_path)

TIFFWriter(data=recons_FDK_cil, file_name=os.path.join(save_path, "out")).write()

In [ ]:
visualisation_window = (0.0, 0.3)
islicer(recons_FDK_cil, direction='vertical', minmax=visualisation_window, origin="upper-left")

In [ ]:
islicer(recons_FDK_cil, direction='horizontal_x', minmax=visualisation_window, origin="upper-left")

## FDK using Tigre filter and Tigre projector

In [ ]:
start = time.time()

print("Filter: Tigre")
print("Projector: Tigre")
# Prepare the data for Astra-toolbox
data_corr.reorder(order='tigre')

# Reconstruct using FDK
ig = data_corr.geometry.get_ImageGeometry()

if roi_size is not None:
    ig.voxel_num_x = roi_size[0]
    ig.voxel_num_y = roi_size[1]
    ig.voxel_num_z = roi_size[2]

reconstruction_algorithm =  FBP_tigre(ig, data_corr.geometry)
recons_FDK_tigre = reconstruction_algorithm(data_corr)

stop = time.time()
runtime, unit = getRuntime(start, stop)
print("Execution time:", "{0:0.2f}".format(runtime), unit)

In [ ]:
save_base_path = os.getcwd()
save_path = os.path.join(path, "recons_FDK_tigre")
print("Print the CT data will be saved in:", save_path)

if not os.path.isdir(save_path):
    os.makedirs(save_path)

TIFFWriter(data=recons_FDK_tigre, file_name=os.path.join(save_path, "out")).write()

## FDK using Astra filter and Astra projector

In [ ]:
start = time.time()

print("Filter: Astra-toolbox")
print("Projector: Astra-toolbox")
# Prepare the data for Astra-toolbox
data_corr.reorder(order='astra')

# Reconstruct using FDK
ig = data_corr.geometry.get_ImageGeometry()

if roi_size is not None:
    ig.voxel_num_x = roi_size[0]
    ig.voxel_num_y = roi_size[1]
    ig.voxel_num_z = roi_size[2]

reconstruction_algorithm =  FBP_astra(ig, data_corr.geometry)
recons_FDK_astra = reconstruction_algorithm(data_corr)

stop = time.time()
runtime, unit = getRuntime(start, stop)
print("Execution time:", "{0:0.2f}".format(runtime), unit)

In [ ]:
save_base_path = os.getcwd()
save_path = os.path.join(path, "recons_FDK_astra")
print("Print the CT data will be saved in:", save_path)

if not os.path.isdir(save_path):
    os.makedirs(save_path)

TIFFWriter(data=recons_FDK_astra, file_name=os.path.join(save_path, "out")).write()

## Compare the two reconstructions using synchronised views

In [ ]:
sl1 = islicer(recons_FDK_cil,   minmax=visualisation_window, origin="upper-left", title="Filter: CIL\nProjection: Tigre\nDirection vertical: Slice")
sl2 = islicer(recons_FDK_tigre, minmax=visualisation_window, origin="upper-left", title="Filter: Tigre\nProjection: Tigre\nDirection vertical: Slice")
sl3 = islicer(recons_FDK_astra, minmax=visualisation_window, origin="upper-left", title="Filter: Astra-toolbox\nProjection: Astra-toolbox\nDirection vertical: Slice")
link_islicer(sl1, sl2, sl3)

In [ ]:
# import SimpleITK as sitk
# reconstruction_as_array = recons_FDK_astra.as_array()
# sitk_image = sitk.GetImageFromArray(reconstruction_as_array)
# sitk.WriteImage(sitk_image, "temp.mha", useCompression=True)

## Iterative reconstruction

Create the projector

In [ ]:
data_corr.reorder(order='astra')
ig = data_corr.geometry.get_ImageGeometry()
ag = data_corr.geometry.copy()
A = ProjectionOperator(ig, ag, device="gpu")

Create the initial guess

In [ ]:
x0 = ig.allocate(0.0)


## SIRT reconstruction

In [ ]:
mysirt_lower0 = SIRT(initial=x0,
                     operator=A,
                     data=data_corr,
                     max_iteration=1000,
                     lower=0.0,
                     update_objective_interval=50)

In [ ]:
mysirt_lower0.run(5, verbose=1)

In [ ]:
islicer(mysirt_lower0.solution, origin="upper-left", direction='vertical', minmax=visualisation_window)

In [ ]:
mysirt_lower0.run(10, verbose=1)
islicer(mysirt_lower0.solution, origin="upper-left", direction='vertical', minmax=visualisation_window)

## CGLS reconstruction

In [ ]:
cgls = CGLS(initial=x0, 
            operator=A, 
            data=data_corr,
            max_iteration = 10,
            lower=0.0,
            update_objective_interval = 1 )

In [ ]:
cgls.run(5, verbose=True)
islicer(cgls.solution, origin="upper-left", direction='vertical', minmax=visualisation_window)

In [ ]:
cgls.run(100, verbose=1)
islicer(cgls.solution, origin="upper-left", direction='vertical', minmax=visualisation_window)